In [1]:
# Import Dependencies
import pandas as pd
import os

In [2]:
# Load file, read and display with Pandas
filepath = os.path.join('.', 'Resources', 'purchase_data.csv')
purchase_df = pd.read_csv(filepath)
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Find number of unique players
player_count = len(pd.unique(purchase_df["SN"]))
total_players_df = pd.DataFrame({"Total Players": [player_count]})

# Display Final Output
total_players_df

,Total Players
0,576


In [4]:
# Quick check of the data for future reference
purchase_df.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [5]:
# Perform Calculations 
unique_items = len(pd.unique(purchase_df["Item ID"])) 
avg_price = purchase_df["Price"].mean()
purchase_num = len(pd.unique(purchase_df["Purchase ID"]))
total_rev = purchase_df["Price"].sum()

# Create Dataframe
purchase_analysis_df = pd.DataFrame({
    "Number of Unique Items": [unique_items],
    "Average Price": "${:,.2f}".format(avg_price), 
    "Number of Purchases": [purchase_num],
    "Total Revenue": "${:,.2f}".format(total_rev)
})

# Display Final Output
purchase_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [6]:
# Get rid of duplicates
clean_purchase_df = purchase_df.drop_duplicates("SN")
total_genders_df = clean_purchase_df.loc[:, ["Gender", "SN", "Age"]]

# Calculate count
gender_values = total_genders_df["Gender"].value_counts()

# Create dataframe with count
total_genders_df = pd.DataFrame([gender_values]).transpose()

# Calculate percentage for gender count and format
gender_percent = total_genders_df["Gender"]/player_count
gender_percent = (100 * (gender_percent).round(4)).astype(str) + "%"
total_genders_df["Percentage of Players"] = gender_percent

# Display final output
total_genders_df

,Gender,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [7]:
# Perform calculations
count_gender = purchase_df.groupby(["Gender"]).count()["Purchase ID"].rename("Purchase Count")
avg_price_gender = purchase_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
total_purchase_gender = purchase_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
avg_total_gender = purchase_df.groupby(["Gender"]).sum()["Price"]/gender_values.rename("Average Total Purchase per Person")

#Create dataframe for gender data
gender_data_df = pd.DataFrame({
    "Purchase Count": count_gender,
    "Average Purchase Price": avg_price_gender,
    "Total Purchase Value": total_purchase_gender,
    "Average Total Purchase per Person": avg_total_gender
})

#Formatting numbers
gender_data_df["Average Purchase Price"] = gender_data_df["Average Purchase Price"].map("${:,.2f}".format)
gender_data_df["Total Purchase Value"] = gender_data_df["Total Purchase Value"].map("${:,.2f}".format)
gender_data_df["Average Total Purchase per Person"] = gender_data_df["Average Total Purchase per Person"].map("${:,.2f}".format)

# Display final output
gender_data_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [8]:
# Add bins for age groups
bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Extract data and group by bins
clean_purchase_df["Bin"] = pd.cut(clean_purchase_df["Age"], bins, labels=group_names)
age_group = clean_purchase_df.groupby("Bin")

# Perform calculations
count_age = age_group["Bin"].count()
percentage_players = (100 * ((age_group["Bin"].count())/player_count))

#Create dataframe
age_group_df = pd.DataFrame({
    "Total Count": count_age,
    "Percentage of Players": percentage_players
})

# Formatting numbers
age_group_df["Percentage of Players"] = age_group_df["Percentage of Players"].map("{:,.2f}%".format)

# Display final output
age_group_df

C:\Users\kimso\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Total Count,Percentage of Players
Bin,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [9]:
# Use binned information to get data
purchase_df["Age Ranges"] = pd.cut(purchase_df["Age"], bins, labels = group_names)
age_analysis =  purchase_df.groupby("Age Ranges")

# Perform calculations
count_age_range = age_analysis["Age Ranges"].count()
avg_purchase_age = age_analysis["Price"].mean()
total_purchase_age = age_analysis["Price"].sum()
avg_purchase_person = age_analysis["Price"].sum() / age_group_df["Total Count"]

#Create dataframe
age_analysis_df = pd.DataFrame({
    "Purchase Count": count_age_range,
    "Average Purchase Price": avg_purchase_age,
    "Total Purchase Value": total_purchase_age,
    "Average Total Purchase per Person": avg_purchase_person
})

# Formatting numbers
age_analysis_df["Average Purchase Price"] = age_analysis_df["Average Purchase Price"].map("${:,.2f}".format)
age_analysis_df["Total Purchase Value"] = age_analysis_df["Total Purchase Value"].map("${:,.2f}".format)
age_analysis_df["Average Total Purchase per Person"] = age_analysis_df["Average Total Purchase per Person"].map("${:,.2f}".format)

# Display final output
age_analysis_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [10]:
# Perform calculations
count_sn = purchase_df.groupby(["SN"]).count()["Price"].rename("Purchase Count")
average_price_sn = purchase_df.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
total_purchase_sn = purchase_df.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")

# Convert to dataFrame
sn_data_df = pd.DataFrame({
    "Purchase Count": count_sn,
    "Average Purchase Price": average_price_sn,
    "Total Purchase Value": total_purchase_sn
})

# Sort by descending
sn_data_df = sn_data_df.sort_values("Total Purchase Value", ascending=False)

# Formatting Numbers
sn_data_df["Average Purchase Price"] = sn_data_df["Average Purchase Price"].map("${:,.2f}".format)
sn_data_df["Total Purchase Value"] = sn_data_df["Total Purchase Value"].map("${:,.2f}".format)

# Display final output
sn_data_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [11]:
# Extract data
most_pop = purchase_df.loc[:,["Item ID", "Item Name", "Price"]]

# Perform calculations
count_pop = most_pop.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")
average_price_pop = most_pop.groupby(["Item ID", "Item Name"]).mean()["Price"]
total_purchase_pop = most_pop.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")

# Create dataframe for Most Popular Items
most_pop_df = pd.DataFrame({
    "Purchase Count": count_pop, 
    "Item Price": average_price_pop,
    "Total Purchase Value": total_purchase_pop
})

# Sort values
sorted_most_pop_df = most_pop_df.sort_values("Purchase Count", ascending = False)

# Formatting numbers
sorted_most_pop_df["Purchase Count"] = sorted_most_pop_df["Purchase Count"]
sorted_most_pop_df["Item Price"] = sorted_most_pop_df["Item Price"].map("${:,.2f}".format)
sorted_most_pop_df["Total Purchase Value"] = sorted_most_pop_df["Total Purchase Value"].map("${:,.2f}".format)

# Display final output
sorted_most_pop_df.head(20)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
75,Brutality Ivory Warmace,8,$2.42,$19.36
103,Singed Scalpel,8,$4.35,$34.80
34,Retribution Axe,8,$2.22,$17.76


In [12]:
# Sort values
sorted_most_prof_df = most_pop_df.sort_values("Total Purchase Value", ascending = False)

#Formatting numbers
sorted_most_prof_df["Purchase Count"] = sorted_most_prof_df["Purchase Count"]
sorted_most_prof_df["Item Price"] = sorted_most_prof_df["Item Price"].map("${:,.2f}".format)
sorted_most_prof_df["Total Purchase Value"] = sorted_most_prof_df["Total Purchase Value"].map("${:,.2f}".format)

# Display final output
sorted_most_prof_df.head(20)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16


# Three Observations about Heroes of Pymoli

I'm going to discuss three take-aways from the data above about the revenue from the Heroes of Pymoli game. We'll examine gender, age and price of items sold in-game to see how these factors impact the game's profitability.

### The Impact of Gender on Revenue

Heroes of Pymoli players are mostly male, making up 84% of all players. They also make 84% of total purchases and account for 83% of all money earned. Females make up most of the remaining players with 14%, and the remaining 2% of players are in the other/non-disclosed category. Females make 14% of total purchases and 15% of total money made. Those in the other/non-disclosed category likewise spend according to their representation by contributing 2% of purchases and revenue.

These numbers suggest that all genders are equally as likely to spend. The most profitable gender group is most profitable simply because it is the largest group. Given this information, a couple ways we could possibly increase revenue would be to 1), focus on the items most popular with males and make those items more enticing or 2), market to recruit a larger female customer base.

### The Impact of Age on Revenue

Nearly half (45%) of all players are between 19 and 24 years old. Those players also account for 47% of total purchases and total revenue. The 24% of players younger than this age group bring in 24% of total revenue, and the 29% of older players bring in 29% of total revenue. As in our gender findings, we see that the spending across age groups aligns with their representation in number of players. It may be wise to explore whether the reasons for making purchases are the same across age groups or whether different age groups tend to purchase certain items.

### The (Lack of) Impact of Price on Purchasing Decisions

Interestingly, there doesn't appear to be much relationship between price of an item and its popularity, according to the Most Popular data above. (It would be beneficial to see the price and purchase count in a scatter plot to know this for sure.) So if price is not a significant influence in popularity of an item, there is another factor that customers are considering when making their purchasing decisions. It may help to categorize the items by type to see if there is a common feature that make certain items more popular.